In [19]:
import numpy as np
from PIL import Image
import time, re, os, sys
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.distance import pdist,squareform

In [20]:
FOLDER = ''
POINT_NUM = 10000
LENGTH = 100
CLUSTER_NUM = 2
GIF_path = './GIF'
colormap = ['black', 'darkorange', 'cornflowerblue', 'silver', 
	'gold', 'green', 'navy', 'magenta', 'yellow', 'red', 'peru', 'pink', 
	'dodgerblue', 'greenyellow', 'cyan']

In [21]:
def openImage(path):
	image = cv2.imread(path)
	H, W, C = image.shape
	image_flat = np.zeros((W * H, C))
	for h in range(H):
		image_flat[h * W:(h + 1) * W] = image[h]

	return image_flat,H,W

In [22]:
def precomputed_kernel(X, gamma_s, gamma_c):
    n=len(X)
    # S(x) spacial information
    S=np.zeros((n,2))
    for i in range(n):
        S[i]=[i//100,i%100]
    print(pdist(S,'sqeuclidean').shape)
    K=squareform(np.exp(-gamma_s*pdist(S,'sqeuclidean')))*squareform(np.exp(-gamma_c*pdist(X,'sqeuclidean')))
    print(K.shape)
    return K


In [23]:
def kernelkmeans(path):
    image_flat, H, W = openImage(path)
    gamma_s = 0.001
    gamma_c = 0.001
    Gram = precomputed_kernel(image_flat, gamma_s, gamma_c)
    # kmeans++ init
    Cluster = np.zeros((CLUSTER_NUM, Gram.shape[1]))
    Cluster[0]=Gram[np.random.randint(low=0,high=Gram.shape[0],size=1),:]
    for c in range(1,CLUSTER_NUM):
            Dist=np.zeros((len(Gram),c))
            for i in range(len(Gram)):
                for j in range(c):
                    Dist[i,j]=np.sqrt(np.sum((Gram[i]-Cluster[j])**2))
            Dist_min=np.min(Dist,axis=1)
            sum=np.sum(Dist_min)*np.random.rand()
            for i in range(len(Gram)):
                sum-=Dist_min[i]
                if sum<=0:
                    Cluster[c]=Gram[i]
                    break


In [24]:
kernelkmeans('image1.png')

(49995000,)
(10000, 10000)
